<a href="https://colab.research.google.com/github/Yekumo/video-captioning-deep-learning/blob/main/Video_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print("PyTorch Version:", torch.__version__)
print("GPU Var mı?:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Adı:", torch.cuda.get_device_name(0))
else:
    print("GPU AKTİF DEĞİL! Ayarları kontrol et.")

PyTorch Version: 2.9.0+cu126
GPU Var mı?: True
GPU Adı: Tesla T4


In [7]:
# 1. Google Drive'ı Bağla (İzin isteyecek, onayla)
from google.colab import drive
import os

drive.mount('/content/drive')

# 2. Proje için Drive içinde düzenli bir klasör oluşturalım
# Bu klasör senin Drive'ında "Video_Captioning_Proje" adıyla oluşacak.
project_path = '/content/drive/My Drive/Video_Captioning_Proje'
dataset_path = os.path.join(project_path, 'data')

# Klasör yoksa oluştur, varsa uyarı ver
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)
    print(f"✅ Klasörler başarıyla oluşturuldu: {dataset_path}")
else:
    print(f"ℹ️ Klasör zaten mevcut: {dataset_path}")

# Çalışma dizinimizi bu yeni klasör yapalım
os.chdir(project_path)
print(f"📂 Güncel çalışma dizini: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ℹ️ Klasör zaten mevcut: /content/drive/My Drive/Video_Captioning_Proje/data
📂 Güncel çalışma dizini: /content/drive/My Drive/Video_Captioning_Proje


In [2]:
import os

# 1. Önce oluşturduğumuz veri klasörüne gidelim
# (%cd komutu "change directory" demektir, o klasörün içine girer)
%cd /content/drive/My Drive/Video_Captioning_Proje/data

print("📂 Şu anki klasör:", os.getcwd())

# 2. Video Arşivini İndir (YouTubeClips.tar)
# Eğer dosya zaten varsa tekrar indirmez.
if not os.path.exists('YouTubeClips.tar'):
    print("⏳ Video arşivi indiriliyor... (Yaklaşık 1.7 GB)")
    # !wget komutu Linux tabanlı sistemlerde dosya indirmeye yarar
    !wget --no-check-certificate https://www.cs.utexas.edu/users/ml/clamp/videoDescription/YouTubeClips.tar
    print("✅ Video arşivi indi.")
else:
    print("ℹ️ Video arşivi zaten mevcut, indirmeye gerek yok.")

# 3. Arşivi Klasöre Çıkar (Unzip işlemi)
if not os.path.exists('YouTubeClips'):
    print("⏳ Arşiv açılıyor... (Videolar çıkarılıyor)")
    # tar -xvf komutu .tar uzantılı dosyaları açar
    !tar -xvf YouTubeClips.tar > /dev/null 2>&1  # Çıktıyı gizle ki ekran dolmasın
    print("✅ Videolar başarıyla çıkarıldı!")
else:
    print("ℹ️ Videolar zaten klasöre çıkarılmış.")

# 4. Etiket Dosyasını (Metinleri) İndir
# Bu dosya hangi videoda ne olduğunu anlatan cümleleri içerir (CSV formatında)
if not os.path.exists('video_corpus.csv'):
    print("⏳ Etiket dosyası (Descriptions) indiriliyor...")
    !wget --no-check-certificate https://raw.githubusercontent.com/guoro/msvd-dataset/master/video_corpus.csv
    print("✅ Etiket dosyası indi.")
else:
    print("ℹ️ Etiket dosyası zaten mevcut.")

print("\n🎉 TÜM İNDİRME İŞLEMLERİ TAMAMLANDI!")

/content/drive/My Drive/Video_Captioning_Proje/data
📂 Şu anki klasör: /content/drive/My Drive/Video_Captioning_Proje/data
⏳ Video arşivi indiriliyor... (Yaklaşık 1.7 GB)
--2026-02-11 06:45:45--  https://www.cs.utexas.edu/users/ml/clamp/videoDescription/YouTubeClips.tar
Resolving www.cs.utexas.edu (www.cs.utexas.edu)... 128.83.139.191
Connecting to www.cs.utexas.edu (www.cs.utexas.edu)|128.83.139.191|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cs.utexas.edu/~ml/clamp/videoDescription/YouTubeClips.tar [following]
--2026-02-11 06:45:46--  https://www.cs.utexas.edu/~ml/clamp/videoDescription/YouTubeClips.tar
Connecting to www.cs.utexas.edu (www.cs.utexas.edu)|128.83.139.191|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1856460800 (1.7G) [application/x-tar]
Saving to: ‘YouTubeClips.tar’

YouTubeClips.tar    100%[===================>]   1.73G  58.3MB/s    in 30s     

2026-02-11 06:46:16 (58.9 MB/s) - ‘You